In [ ]:
# Print pokemons

api: https://github.com/rubenvereecken/pokemongo-api

In [ ]:
import print_pokemons as pp
from pogo.pokedex import pokedex

In [ ]:
username, password, auth_type = pp.get_credentials_from_keyfile('keyfile')
print(username)

In [ ]:
#username, password, auth_type = pp.get_credentials_from_keyfile('keyfile.cat')
print(username)

In [ ]:
trainer = pp.get_trainer(username, password, auth_type)

In [ ]:
inventory = trainer.session.getInventory()
bag = pp.get_pokemons(trainer)

In [ ]:
pp.print_pokemons_by_iv(bag)

In [ ]:
pp.print_pokemons_by_number(bag)

In [ ]:
pp.print_pokemons_by_cp(bag)

In [ ]:
def print_pokemons_by_number(lst):
    lst.sort(key=lambda x: (x['id'],x['%'], x['cp']))

    sep = '-'*(110)
    print('{:>3} | {:>15} | {:>4} | {:>4} | {:>8} | {:>8} | {:>8} | {:>8} | {:>8} | {:>8} | {}'.format(
        'id', 'name', 'cp', 'hp', 'attack', 'defense', 'stamina', '%', 'candies', 'n_evolves', 'family'))
    print(sep)

    curr_family = ''
    counter = 0

    for (i, p) in enumerate(lst, start=1):
        if not curr_family:
            curr_family=p['family']

        if p['family'] != curr_family:
            curr_family=p['family']
            print('')

        print('{id:>3} | {name:>15} | {cp:>4} | {hp:>4} | {attack:>8} | {defense:>8} | {stamina:>8} | {%:>8.3f} | {candies:>8} | {n_evolves:>8} | {family}'.format(**p))

    print('Sub total: {}'.format(counter))
    print('Pokemons Total: {}'.format(len(lst)))
    

In [ ]:
print_pokemons_by_number( bag )

## Count pokemons and possible evolves

In [ ]:
from collections import Counter

def print_most_common(bag, limit=None):
    counter = Counter([(x['name'], x['id']) for x in bag if pokedex.evolves.get(x['id'], 0) > 0])

    
    print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format('Name', 'Need', 'Candies', 'NumHave', 'Can Evo.', 'Curr. Evo.'))
    print('-'*70)

    total_possible = 0
    total_current = 0
    
    total_trio_possible = 0
    total_trio_current = 0
    
    if limit is None:
        most_common = sorted(counter.most_common())
    else:
        most_common = sorted(counter.most_common(limit))

    for item, count in most_common:
        family = pokedex.families[item[1]]

        n_candies = inventory.candies.get(item[1], 0)
        need = pokedex.evolves.get(item[1], 0)

        possible = int(n_candies / need)
        curr = min(possible, count)
        
        if possible == 0:
            continue
        print('{:>15} | {:>5} | {:>7} | {:>7} | {:>8} | {}'.format(item[0], need, n_candies, count, possible, curr))

        total_possible += possible
        total_current += curr
        
        if ( item[0] == 'CATERPIE' or item[0] == 'PIDGEY' or item[0] == 'WEEDLE'):
            total_trio_possible += possible
            total_trio_current += curr

    print('\nTotal possible evolutions: {}'.format(total_possible))
    print('Current total evolutions: {}'.format(total_current))
    
    print('\nTotal TRIO possible evolutions: {}'.format(total_trio_possible))
    print('Current TRIO total evolutions: {}'.format(total_trio_current))
    

In [ ]:
print_most_common(bag)

In [ ]:
from collections import Counter

def print_pokemons_by_number_totals(lst):
    lst.sort(key=lambda x: (x['id'],x['%'], x['cp']))

    counter = Counter([(x['name'], x['id']) for x in bag if pokedex.evolves.get(x['id'], 0) > 0])

    most_common = sorted(counter.most_common())
   # print(lst)

    for (i, p) in enumerate(lst, start=1):
        if 'num_have' in p:
            continue
            
        for item, count in most_common:
            if item[0] == p['name']:
                p['num_have'] = count
                break
                
        if 'num_have' not in p:
            p['num_have'] = 0
                
        
    sep = '-'*(110)
    print('{:>3} | {:>15} | {:>4} | {:>8} | {:>8} | {:>5} |{:>5} |  {}'.format(
        'id', 'name', 'cp', '%', 'candies', 'can_ev','have', 'family'))
    print(sep)

    curr_family = ''
    counter = 0

    for (i, p) in enumerate(lst, start=1):
        if not curr_family:
            curr_family=p['family']

        if p['family'] != curr_family:
            curr_family=p['family']
            print('')

        print('{id:>3} | {name:>15} | {cp:>4} | {%:>8.3f} | {candies:>8} | {n_evolves:>5} |{num_have:>5} | {family}'.format(**p))

    print('Sub total: {}'.format(counter))
    print('Pokemons Total: {}'.format(len(lst)))
    

In [ ]:
inventory = trainer.session.getInventory()
bag = pp.get_pokemons(trainer)
print_pokemons_by_number_totals(bag)